# Model Export
1) Re-Build generator architecture
2) Reload trained weights from checkpoint
3) export with ONNX

### 1) Model Reinitialization
Copy from Model Build/Train file

In [1]:
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def attention_block(x, filters):
    """Self-attention mechanism with dynamic input support"""
    shape = tf.shape(x)
    batch_size, height, width = shape[0], shape[1], shape[2]
    
    f = tf.keras.layers.Conv2D(filters // 8, 1)(x)
    g = tf.keras.layers.Conv2D(filters // 8, 1)(x)
    h = tf.keras.layers.Conv2D(filters, 1)(x)
    
    f = tf.reshape(f, [batch_size, -1, filters // 8])
    g = tf.reshape(g, [batch_size, -1, filters // 8])
    h = tf.reshape(h, [batch_size, -1, filters])
    
    s = tf.matmul(f, g, transpose_b=True)
    beta = tf.nn.softmax(s)
    
    o = tf.matmul(beta, h)
    o = tf.reshape(o, [batch_size, height, width, filters])
    
    gamma = tf.Variable(0.0, trainable=True)
    return x + gamma * o

def spectral_conv2d(filters, kernel_size, strides=1):
    return tf.keras.layers.Conv2D(
        filters,
        kernel_size,
        strides=strides,
        padding='same',
        kernel_initializer='orthogonal',
        use_bias=False
    )

def downsample(filters, size, apply_batchnorm=True):
    result = tf.keras.Sequential()
    result.add(spectral_conv2d(filters, size, strides=2))
    
    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    
    result.add(tf.keras.layers.PReLU(shared_axes=[1, 2]))
    return result

def upsample(filters, size, apply_dropout=False):
    result = tf.keras.Sequential()
    result.add(
        tf.keras.layers.Conv2DTranspose(
            filters, size, strides=2,
            padding='same',
            kernel_initializer='orthogonal',
            use_bias=False
        )
    )
    
    result.add(tf.keras.layers.LayerNormalization())
    
    if apply_dropout:
        result.add(tf.keras.layers.SpatialDropout2D(0.5))
    
    result.add(tf.keras.layers.PReLU(shared_axes=[1, 2]))
    return result

def resnet_block(input_tensor, filters, kernel_size=3):
    def squeeze_excite_block(input_tensor, ratio=16):
        channels = input_tensor.shape[-1]
        
        se = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
        se = tf.keras.layers.Reshape((1, 1, channels))(se)
        se = tf.keras.layers.Dense(channels // ratio, activation='relu')(se)
        se = tf.keras.layers.Dense(channels, activation='sigmoid')(se)
        
        return tf.keras.layers.Multiply()([input_tensor, se])
    
    x = spectral_conv2d(filters, kernel_size)(input_tensor)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.PReLU(shared_axes=[1, 2])(x)
    
    x = spectral_conv2d(filters, kernel_size)(x)
    x = tf.keras.layers.LayerNormalization()(x)
    
    x = squeeze_excite_block(x)
    x = tf.keras.layers.Add()([x, input_tensor])
    x = tf.keras.layers.PReLU(shared_axes=[1, 2])(x)
    
    return x

2024-12-05 21:21:45.183186: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 21:21:45.183285: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 21:21:45.183510: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 21:21:45.215172: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import tensorflow as tf
from tensorflow.keras.utils import register_keras_serializable

@register_keras_serializable(package="CustomLayers")
class DynamicPaddingLayer(tf.keras.layers.Layer):
    def __init__(self, target_multiple=8, **kwargs):
        super(DynamicPaddingLayer, self).__init__(**kwargs)
        self.target_multiple = target_multiple
        self.built = False
        
    def build(self, input_shape):
        # No dynamic padding weights needed for fixed 64x64 input
        self.built = True
        super(DynamicPaddingLayer, self).build(input_shape)
        
    def call(self, inputs):
        # Fixed padding for 64x64 input to match a multiple of 8
        paddings = tf.convert_to_tensor([
            [0, 0],  # Batch dimension
            [0, 0],  # Height (no padding needed for 64)
            [0, 0],  # Width (no padding needed for 64)
            [0, 0]   # Channels dimension
        ], dtype=tf.int32)
        
        return tf.pad(inputs, paddings, mode='REFLECT')
    
    def get_config(self):
        config = super(DynamicPaddingLayer, self).get_config()
        config.update({
            'target_multiple': self.target_multiple
        })
        return config

@register_keras_serializable(package="CustomLayers")
class UnpadLayer(tf.keras.layers.Layer):
    def __init__(self, padding_layer=None, **kwargs):
        super(UnpadLayer, self).__init__(**kwargs)
        
    def call(self, inputs):
        # No unpadding needed for 64x64 input
        return inputs
    
    def get_config(self):
        return super(UnpadLayer, self).get_config()

In [6]:
def build_generator(input_shape=(None, None, 3)):
    """Build generator with support for arbitrary input sizes"""
    tf.keras.backend.set_floatx('float32')
    
    # Input layer
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # Add padding layer
    padding_layer = DynamicPaddingLayer(target_multiple=8)
    x = padding_layer(inputs)
    
    # Initial convolution
    x = spectral_conv2d(32, 7)(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.PReLU(shared_axes=[1, 2])(x)
    
    # Downsampling path
    down_stack = [
        downsample(64, 3, apply_batchnorm=False),
        downsample(128, 3),
        downsample(256, 3),
    ]
    
    # Store skip connections
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    
    # ResNet blocks
    num_res_blocks = 9
    for i in range(num_res_blocks):
        x = resnet_block(x, filters=256)
        if i % 2 == 0:
            x = attention_block(x, filters=256)
    
    # Upsampling path
    up_stack = [
        upsample(128, 3, apply_dropout=True),
        upsample(64, 3),
        upsample(32, 3),
    ]
    
    # Decoder with skip connections
    skips = reversed(skips[:-1])
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = attention_block(x, x.shape[-1])
        x = tf.keras.layers.Concatenate()([x, skip])
    
    # Final output layer
    x = tf.keras.layers.Conv2DTranspose(
        3, 4,
        strides=2,
        padding='same',
        kernel_initializer='orthogonal',
        activation='tanh'
    )(x)
    
    # Remove padding using custom unpad layer
    outputs = UnpadLayer(padding_layer)(x)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)

### 2) Reload trained weights
from checkpoint exported from training file

In [ ]:
import tensorflow as tf
import os

# Define the checkpoint directory
checkpoint_dir = './training_checkpointsH'

# Initialize generator model
generator = build_generator(input_shape=(None, None, 3)) 

# Create a checkpoint object for the generator only
checkpoint = tf.train.Checkpoint(generator=generator)

# Restore the latest checkpoint
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)

if latest_checkpoint:
    checkpoint.restore(latest_checkpoint).expect_partial()
    print(f"Checkpoint restored from {latest_checkpoint}")
else:
    # Debug: Print contents of the directory
    print("Contents of checkpoint directory:")
    print(os.listdir(checkpoint_dir))
    print("No checkpoint found.")

# Export saving generator
generator.save('./models/pressure_predict_H.keras')

### 3) ONNX export
convert to onnx framework and export to disk to be evaluated in Houdini

In [ ]:
import tensorflow as tf
import tf2onnx
import onnx

def export_to_onnx(model_path, output_path):
    """
    Helper function to export saved model to ONNX format with dynamic input shapes
    Handles conversion from NHWC (TensorFlow) to NCHW (ONNX) format while preserving padding
    """
    
    # Load the saved model
    model = tf.keras.models.load_model(model_path)
    
    # Create a wrapper model that handles the NHWC->NCHW conversion explicitly
    input_layer = tf.keras.layers.Input(shape=(None, None, 3), name="input_nhwc")
    
    # Convert NHWC to NCHW before model
    x = tf.transpose(input_layer, [0, 3, 1, 2], name="to_nchw")
    
    # Run through model
    output = model(x)
    
    # Convert back to NHWC for proper padding handling
    output = tf.transpose(output, [0, 2, 3, 1], name="to_nhwc")
    
    wrapper_model = tf.keras.Model(inputs=input_layer, outputs=output)
    
    # Use NCHW format with dynamic spatial dimensions
    input_signature = [tf.TensorSpec([None, None, None, 3], tf.float32, name="input")]
    
    # Convert to ONNX without automatic NCHW conversion
    model_proto, external_tensor_storage = tf2onnx.convert.from_keras(
        wrapper_model, 
        input_signature=input_signature,
        opset=13
    )
    
    # Function to set shape parameters
    def set_shape_params(tensor):
        shape = tensor.type.tensor_type.shape
        # Make batch dimension dynamic
        shape.dim[0].dim_value = 0
        shape.dim[0].dim_param = "batch"
        # Set channel dimension (3 for RGB)
        shape.dim[1].dim_value = 3
        # Make spatial dimensions dynamic
        shape.dim[2].dim_value = 0
        shape.dim[2].dim_param = "height"
        shape.dim[3].dim_value = 0
        shape.dim[3].dim_param = "width"
    
    # Modify input shapes
    for tensor in model_proto.graph.input:
        set_shape_params(tensor)
    
    # Modify output shapes
    for tensor in model_proto.graph.output:
        set_shape_params(tensor)
    
    # Save the ONNX model
    onnx.save(model_proto, output_path)
    
    # Verify the exported model
    onnx_model = onnx.load(output_path)
    onnx.checker.check_model(onnx_model)
    
    # Print model input/output shapes
    print(f"Model exported to {output_path}")
    print("\nInput shapes:")
    for inp in onnx_model.graph.input:
        print(f"{inp.name}: {[d.dim_param if d.dim_param else d.dim_value for d in inp.type.tensor_type.shape.dim]}")
    print("\nOutput shapes:")
    for out in onnx_model.graph.output:
        print(f"{out.name}: {[d.dim_param if d.dim_param else d.dim_value for d in out.type.tensor_type.shape.dim]}")

# Usage
model_path = "./models/pressure_predict_H.keras"
output_path = "./models/pressure_predict_H3.onnx"
export_to_onnx(model_path, output_path)

In [ ]:
import tensorflow as tf
import tf2onnx
import onnx

def export_to_onnx_static(model_path, output_path):
    """
    Helper function to export saved model to ONNX format with fixed 64x64 input shape
    Handles conversion from NHWC (TensorFlow) to NCHW (ONNX) format while preserving padding
    """
    
    # Load the saved model
    model = tf.keras.models.load_model(model_path)
    
    # Create a wrapper model that handles the NHWC->NCHW conversion explicitly
    # Now with a fixed input shape of 64x64
    input_layer = tf.keras.layers.Input(shape=(64, 64, 3), name="input_nhwc")
    
    # Convert NHWC to NCHW before model
    x = tf.transpose(input_layer, [0, 3, 1, 2], name="to_nchw")
    
    # Run through model
    output = model(x)
    
    # Convert back to NHWC for proper padding handling
    output = tf.transpose(output, [0, 2, 3, 1], name="to_nhwc")
    
    wrapper_model = tf.keras.Model(inputs=input_layer, outputs=output)
    
    # Use NCHW format with fixed 64x64 spatial dimensions
    input_signature = [tf.TensorSpec([None, 64, 64, 3], tf.float32, name="input")]
    
    # Convert to ONNX without automatic NCHW conversion
    model_proto, external_tensor_storage = tf2onnx.convert.from_keras(
        wrapper_model, 
        input_signature=input_signature,
        opset=13
    )
    
    # Function to set shape parameters with fixed 64x64 dimensions
    def set_shape_params(tensor):
        shape = tensor.type.tensor_type.shape
        # Make batch dimension dynamic
        shape.dim[0].dim_value = 0
        shape.dim[0].dim_param = "batch"
        # Set channel dimension (3 for RGB)
        shape.dim[1].dim_value = 3
        # Set fixed spatial dimensions to 64
        shape.dim[2].dim_value = 64
        shape.dim[3].dim_value = 64
        # Remove any dynamic parameters for height and width
        if len(shape.dim) > 2:
            if hasattr(shape.dim[2], 'dim_param'):
                del shape.dim[2].dim_param
            if hasattr(shape.dim[3], 'dim_param'):
                del shape.dim[3].dim_param
    
    # Modify input shapes
    for tensor in model_proto.graph.input:
        set_shape_params(tensor)
    
    # Modify output shapes
    for tensor in model_proto.graph.output:
        set_shape_params(tensor)
    
    # Save the ONNX model
    onnx.save(model_proto, output_path)
    
    # Verify the exported model
    onnx_model = onnx.load(output_path)
    onnx.checker.check_model(onnx_model)
    
    # Print model input/output shapes
    print(f"Model exported to {output_path}")
    print("\nInput shapes:")
    for inp in onnx_model.graph.input:
        print(f"{inp.name}: {[d.dim_param if d.dim_param else d.dim_value for d in inp.type.tensor_type.shape.dim]}")
    print("\nOutput shapes:")
    for out in onnx_model.graph.output:
        print(f"{out.name}: {[d.dim_param if d.dim_param else d.dim_value for d in out.type.tensor_type.shape.dim]}")

# Usage
model_path = "./models/pressure_predict_H.keras"
output_path = "./models/pressure_predict_H_static.onnx"
export_to_onnx_static(model_path, output_path)

In [ ]:
import tensorflow as tf
import tf2onnx
import onnx

def export_to_onnx(model_path, output_path):
    """
    Helper function to export saved model to ONNX format with fixed 64x64 input shape
    """
    
    # Load the saved model
    model = tf.keras.models.load_model(model_path)
    
    # Create a wrapper model that handles the input conversion explicitly
    input_layer = tf.keras.layers.Input(shape=(8, 64, 64), name="input_nchw")
    
    # Run through model directly with NCHW input
    output = model(input_layer)
    
    wrapper_model = tf.keras.Model(inputs=input_layer, outputs=output)
    
    # Use fixed NCHW input shape
    input_signature = [tf.TensorSpec([None, 8, 64, 64], tf.float32, name="input")]
    
    # Convert to ONNX 
    model_proto, external_tensor_storage = tf2onnx.convert.from_keras(
        wrapper_model, 
        input_signature=input_signature,
        opset=13
    )
    
    # Function to set shape parameters with fixed 64x64 dimensions
    def set_shape_params(tensor):
        shape = tensor.type.tensor_type.shape
        # Make batch dimension dynamic
        shape.dim[0].dim_value = 0
        shape.dim[0].dim_param = "batch"
        # Set channel dimension (8)
        shape.dim[1].dim_value = 8
        # Set fixed spatial dimensions to 64
        shape.dim[2].dim_value = 64
        shape.dim[3].dim_value = 64
        # Remove any dynamic parameters
        if len(shape.dim) > 2:
            if hasattr(shape.dim[2], 'dim_param'):
                del shape.dim[2].dim_param
            if hasattr(shape.dim[3], 'dim_param'):
                del shape.dim[3].dim_param
    
    # Modify input shapes
    for tensor in model_proto.graph.input:
        set_shape_params(tensor)
    
    # Modify output shapes
    for tensor in model_proto.graph.output:
        set_shape_params(tensor)
    
    # Save the ONNX model
    onnx.save(model_proto, output_path)
    
    # Verify the exported model
    onnx_model = onnx.load(output_path)
    onnx.checker.check_model(onnx_model)
    
    # Print model input/output shapes
    print(f"Model exported to {output_path}")
    print("\nInput shapes:")
    for inp in onnx_model.graph.input:
        print(f"{inp.name}: {[d.dim_param if d.dim_param else d.dim_value for d in inp.type.tensor_type.shape.dim]}")
    print("\nOutput shapes:")
    for out in onnx_model.graph.output:
        print(f"{out.name}: {[d.dim_param if d.dim_param else d.dim_value for d in out.type.tensor_type.shape.dim]}")

# Usage
model_path = "./models/pressure_predict_H.keras"
output_path = "./models/pressure_predict_H_static.onnx"
export_to_onnx(model_path, output_path)

In [ ]:
import onnxruntime
import numpy as np

def test_onnx_model_with_padding(model_path, sample_batch_size=1, sample_width=64, sample_height=64):
    """
    Test an ONNX model with dynamic padding, checking for dimension handling
    """
    try:
        session = onnxruntime.InferenceSession(model_path, providers=['CPUExecutionProvider'])
        
        # Get model metadata
        input_name = session.get_inputs()[0].name
        input_shape = session.get_inputs()[0].shape
        print(f"Model input name: {input_name}")
        print(f"Model input shape: {input_shape}")
        
        # Test multiple input sizes to check padding behavior
        test_sizes = [
            (sample_height, sample_width),
            (sample_height + 1, sample_width + 1),  # Non-multiple of 8
            (sample_height - 1, sample_width - 1),  # Non-multiple of 8
            (64, 64),  # Multiple of 8
            (72, 72)   # Multiple of 8
        ]
        
        for height, width in test_sizes:
            print(f"\nTesting input size: {height}x{width}")
            sample_input = np.random.randn(sample_batch_size, 3, height, width).astype(np.float32)
            
            # Run inference
            outputs = session.run(None, {input_name: sample_input})
            
            print(f"Input shape: {sample_input.shape}")
            print(f"Output shape: {outputs[0].shape}")
            
            # Check if output dimensions match input or are padded
            if outputs[0].shape[2:] != sample_input.shape[2:]:
                print("⚠️ Dimension mismatch!")
                print(f"Height diff: {outputs[0].shape[2] - sample_input.shape[2]}")
                print(f"Width diff: {outputs[0].shape[3] - sample_input.shape[3]}")
            
            # Check if output dimensions are multiples of 8
            for dim in outputs[0].shape[2:]:
                if dim % 8 != 0:
                    print(f"⚠️ Output dimension {dim} is not a multiple of 8")
        
        print("\nValue statistics for last test case:")
        output = outputs[0]
        print(f"Min value: {output.min()}")
        print(f"Max value: {output.max()}")
        print(f"Mean value: {output.mean()}")
        print(f"Has NaN: {np.isnan(output).any()}")
        print(f"Has Inf: {np.isinf(output).any()}")
            
    except Exception as e:
        print(f"Error testing model: {str(e)}")

# Example usage
if __name__ == "__main__":
    test_onnx_model_with_padding("./models/pressure_predict_H2.onnx")

In [ ]:
asdf asdf

# Modified export code that maintains original model structure
def export_to_onnx(model_path, output_path):
    """Helper function to export saved model to ONNX format"""
    import tf2onnx
    import onnx
    
    # Load the saved model
    model = tf.keras.models.load_model(model_path)
    
    # Convert to ONNX
    input_signature = [tf.TensorSpec([1, None, None, 3], tf.float32, name="input")]
    model_proto, _ = tf2onnx.convert.from_keras(
        model, 
        input_signature=input_signature,
        #opset=13,
        inputs_as_nchw=['input'],
        outputs_as_nchw=['unpad_layer']
    )
    
    # Save the ONNX model
    onnx.save(model_proto, output_path)
    print(f"Model exported to {output_path}")

model_path = "./models/pressure_predict_H.keras"
output_path = "./models/pressure_predict_H_old3.onnx"
export_to_onnx(model_path, output_path)

In [ ]:
asdf asdf
import tf2onnx
# Helper libraries for transposition of NHWC (model) to NCHW (Houdini)
import onnx

# Load model
model = tf.keras.models.load_model(
    "./models/pressure_predict_H.keras",
    custom_objects={
        "DynamicPaddingLayer": DynamicPaddingLayer, 
        "UnpadLayer": UnpadLayer
    }
)

output_path = "./models/pressure_predict_H_old1.onnx"

# Define batch dim
input_shape = (None, 64, 64, 3)  # Add None as batch dimension

# Create input signature with batch dimension
input_signature = [tf.TensorSpec(shape=input_shape, dtype=tf.float32, name='input')]

# Convert the model
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=input_signature, inputs_as_nchw=['input'],  outputs_as_nchw=['conv2d_transpose_5'], opset=13)

# Save the model
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())
# tf2onnx.utils.save_onnx_model(onnx_model, output_path)



In [ ]:
asdf asdf
# Define batch dim
input_shape = (None, 64, 64, 3)  # Add None as batch dimension

# Create input signature with batch dimension
input_signature = [tf.TensorSpec(shape=input_shape, dtype=tf.float32, name='input')]

# Convert to ONNX
onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=input_signature,
    opset=13,
    inputs_as_nchw=['input'],
    #outputs_as_nchw=['output']  # As you mentioned this was the correct setting
)



input_signature = [tf.TensorSpec((None, 64, 64, 3), tf.float32, name='input')]

# Create custom configuration to prevent automatic transpose
extra_opset = []
inputs_as_nchw = ['input']
custom_ops = {}
custom_op_handlers = {}
custom_rewriter = {}

# Set target correctly
target = None  # Let tf2onnx use default target
# Or specify explicitly if needed:
# target = "onnx"

onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=input_signature,
    opset=13,
    inputs_as_nchw=inputs_as_nchw,
    custom_ops=custom_ops,
    custom_op_handlers=custom_op_handlers,
    custom_rewriter=custom_rewriter,
    target=target,
    extra_opset=extra_opset
)

output_path = "./models/pressure_predict_H_old2.onnx"
with open(output_path, "wb") as f:
    f.write(onnx_model.SerializeToString())


print(f"Model successfully converted to ONNX and saved as {output_path}")